## generate ip level target list for all /24 prefixes

In [ ]:
from pathlib import Path
import pickle

from ipaddress import IPv4Network
from random import randint

In [2]:
hitlist_file = Path(".") / "../datasets/internet_address_hitlist_it102w-20230125.fsdb"
verfploeter_hitlist_file = Path(".") / "../datasets/internet_address_verfploeter_hitlist_it102w-20230125.fsdb"

targets_per_prefix = {}
with open(hitlist_file, "r") as f:
    for i, row in enumerate(f.readlines()[1:]):
        row = row.split("\t")

        # get prefix from hex value
        target= row[-1].strip("\n")
        prefix = target.split(".")[:-1]
        prefix.append("0")
        prefix = ".".join(prefix)

        # get targets
        targets = row[-1].strip("\n")
        try: 
            targets_per_prefix[prefix].append(target) 
        except KeyError:
            targets_per_prefix[prefix] = [target]


with open(verfploeter_hitlist_file, "r") as f:
    for i, row in enumerate(f.readlines()[1:]):
        row = row.split("\t")

        # get prefix from hex value
        prefix_hex = row[0]
        prefix = ["".join(x) for x in zip(*[iter(prefix_hex)]*2)]
        prefix = [int(x, 16) for x in prefix]
        prefix = ".".join(str(x) for x in prefix)

        target_list = row[-1].strip("\n")
        target_list = target_list.split(",")

        # parse and save targets
        if target_list[0] != '-' :
            for i, target in enumerate(target_list):
                target_list[i] = prefix.split(".")[:-1]
                target_list[i].append(str(int(target, 16)))
                target_list[i] = ".".join(target_list[i])     

            try: 
                targets_per_prefix[prefix].extend(target_list) 
            except KeyError:
                targets_per_prefix[prefix] = [targets]

      

In [3]:
print("target hitlist")
for i, prefix in enumerate(targets_per_prefix):
    if i > 100: break
    print("prefix:",prefix,"target hitlist:",targets_per_prefix[prefix])  

target hitlist
prefix: 1.0.0.0 target hitlist: ['1.0.0.230', '1.0.0.0', '1.0.0.1', '1.0.0.2']
prefix: 1.0.1.0 target hitlist: ['1.0.1.1']
prefix: 1.0.2.0 target hitlist: ['1.0.2.1']
prefix: 1.0.3.0 target hitlist: ['1.0.3.1']
prefix: 1.0.4.0 target hitlist: ['1.0.4.4', '1.0.4.1', '1.0.4.4']
prefix: 1.0.5.0 target hitlist: ['1.0.5.5', '1.0.5.1', '1.0.5.5']
prefix: 1.0.6.0 target hitlist: ['1.0.6.6', '1.0.6.1', '1.0.6.6']
prefix: 1.0.7.0 target hitlist: ['1.0.7.1', '1.0.7.1', '1.0.7.7']
prefix: 1.0.8.0 target hitlist: ['1.0.8.1']
prefix: 1.0.9.0 target hitlist: ['1.0.9.1']
prefix: 1.0.10.0 target hitlist: ['1.0.10.1']
prefix: 1.0.11.0 target hitlist: ['1.0.11.1']
prefix: 1.0.12.0 target hitlist: ['1.0.12.233']
prefix: 1.0.13.0 target hitlist: ['1.0.13.1']
prefix: 1.0.14.0 target hitlist: ['1.0.14.1']
prefix: 1.0.15.0 target hitlist: ['1.0.15.1']
prefix: 1.0.16.0 target hitlist: ['1.0.16.125', '1.0.16.14', '1.0.16.9', '1.0.16.10', '1.0.16.11']
prefix: 1.0.17.0 target hitlist: ['1.0.17.148

In [4]:
out_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(out_file, "wb") as f:
    pickle.dump(targets_per_prefix, f)

In [5]:
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

In [8]:
for i, anchor_ip in enumerate(anchors):
    nb_ip_addresses = 3
    if i > 10: break
    prefix_anchor = anchor_ip.split(".")[:-1]
    prefix_anchor.append("0")
    prefix_anchor = ".".join(prefix_anchor)
    ip_list = []
    try:
        ip_list = targets_per_prefix[prefix_anchor]
    except KeyError:
        pass

    if len(ip_list) < nb_ip_addresses:

        ip = anchor_ip.split(".")[:2]
        ip.extend(['0','0'])
        ip = ".".join(ip)
        prefix_target = IPv4Network(ip + '/' + str(24))

        ip_list.extend([str(prefix_target[randint(1,254)]) for _ in range(0,nb_ip_addresses - len(ip_list))])
    print(f"ip list for anchor {prefix_anchor}: {ip_list}")

ip list for anchor 213.225.160.0: ['213.225.160.254', '213.225.160.14', '213.225.160.140', '213.225.160.159']
ip list for anchor 145.220.0.0: ['145.220.0.1', '145.220.0.1', '145.220.0.28', '145.220.0.29']
ip list for anchor 5.28.0.0: ['5.28.0.14', '5.28.0.6', '5.28.0.13', '5.28.0.14']
ip list for anchor 193.171.255.0: ['193.171.255.1', '193.171.255.1', '193.171.255.2', '193.171.255.9']
ip list for anchor 192.65.184.0: ['192.65.184.154', '192.65.184.42', '192.65.184.54', '192.65.184.69']
ip list for anchor 185.42.136.0: ['185.42.136.149', '185.42.136.1', '185.42.136.2', '185.42.136.3']
ip list for anchor 197.80.104.0: ['197.80.104.36', '197.80.104.33', '197.80.104.34', '197.80.104.35']
ip list for anchor 200.7.84.0: ['200.7.84.120', '200.7.84.2', '200.7.84.3', '200.7.84.4']
ip list for anchor 76.26.115.0: ['76.26.115.194', '76.26.115.193', '76.26.115.194']
ip list for anchor 194.150.191.0: ['194.150.191.21', '194.150.191.1', '194.150.191.2', '194.150.191.5']
ip list for anchor 199.10.66